In [1]:
import ee
import geemap

In [2]:
ee.Authenticate()

True

In [3]:
ee.Initialize(project="ee-ciut")

In [ ]:
# Filter for target partidos in Buenos Aires
target_partidos = ["La Plata", "Berisso", "Ensenada"]
gaul_l2 = ee.FeatureCollection("projects/sat-io/open-datasets/FAO/GAUL/GAUL_2024_L2")

municipalities = (
    gaul_l2.filter(ee.Filter.eq("gaul0_name", "Argentina"))
    .filter(ee.Filter.eq("gaul1_name", "Buenos Aires"))
    .filter(ee.Filter.inList("gaul2_name", target_partidos))
)

# Load level 1 basins and find intersecting ones
basin_l1 = ee.FeatureCollection(
    "projects/sat-io/open-datasets/HydroAtlas/BasinAtlas/BasinATLAS_v10_lev12"
)
intersecting_basins = basin_l1.filterBounds(municipalities.geometry())
aoi = intersecting_basins.geometry()


# Filter just Argentina tiles (e.g., 2020 only)
population = (
    ee.ImageCollection("WorldPop/GP/100m/pop")
    .filter(ee.Filter.eq("system:index", "ARG_2020"))
    .first()
    .clip(aoi)
)


# GISA Impervious Surface
gisa = (
    ee.ImageCollection("projects/sat-io/open-datasets/GISA_1972_2019")
    .mosaic()
    .clip(aoi)
)


# Wetlands
wetlands = (
    ee.ImageCollection("projects/sat-io/open-datasets/GWL_FCS30")
    .sort("system:time_start", False)
    .first()
    .clip(aoi)
)

river_atlas = ee.FeatureCollection(
    "projects/sat-io/open-datasets/HydroAtlas/RiverAtlas_v10"
).filterBounds(municipalities.geometry())


# Calculate pixel-level vulnerability directly
# This approach skips the administrative unit calculations and NLDI

# Nighttime lights
viirs = ee.ImageCollection("NOAA/VIIRS/DNB/MONTHLY_V1/VCMSLCFG")
ntl_first = viirs.sort("system:time_start").first().clip(aoi)
ntl_composite = viirs.qualityMosaic("avg_rad").clip(aoi)  # \
# .filterDate('2023-01-01', '2024-01-01') \
# .max() \
# .clip(aoi)


# Calculate pixel-level vulnerability with log scaling for better visualization
def calculate_pixel_vulnerability(population_img, nightlights_img):
    """
    Calculate vulnerability score at pixel level with log scaling
    Higher values indicate higher vulnerability
    Returns a single-band image for visualization with population mask
    """
    # Add small constant to population to avoid division by zero
    # Only keep pixels with population > 0
    pop_mask = population_img.gt(0)
    population_masked = population_img.updateMask(pop_mask)

    safe_population = population_masked.add(0.1)

    # Calculate light per person ratio
    light_per_person = nightlights_img.divide(safe_population)

    # Apply log transformation to handle the skewed distribution
    # Add small constant to avoid log(0)
    log_light_per_person = light_per_person.add(0.001).log()

    # Get percentiles for normalization
    percentiles = log_light_per_person.reduceRegion(
        reducer=ee.Reducer.percentile([2, 98]), geometry=aoi, scale=500, maxPixels=1e9
    )

    # Extract min and max values (using 2nd and 98th percentiles for robustness)
    min_val = ee.Number(percentiles.get("avg_rad_p2"))
    max_val = ee.Number(percentiles.get("avg_rad_p98"))

    # Normalize to 0-1 scale
    normalized_lpp = log_light_per_person.subtract(min_val).divide(
        max_val.subtract(min_val)
    )

    # Clamp values to 0-1 range
    normalized_lpp = normalized_lpp.clamp(0, 1)

    # Invert scale so higher values = higher vulnerability
    vulnerability = ee.Image(1).subtract(normalized_lpp)

    # Make sure we have a single band named 'vulnerability'
    vulnerability = vulnerability.select([0], ["vulnerability"])

    return vulnerability


# Calculate the vulnerability score
vulnerability = calculate_pixel_vulnerability(population, ntl_composite)

vulnerability_smoothed = vulnerability.focal_mean(radius=500, units="meters")


In [ ]:
# Create map and center on AOI
Map = geemap.Map(basemap="CartoDB.Positron")
Map.centerObject(aoi, 9)


# population (2020)
Map.addLayer(
    population,
    {
        "bands": ["population"],
        "min": 0.0,
        "max": 50.0,
        "palette": ["24126c", "1fff4f", "d4ff50"],
    },
    "WorldPop Population 100m",
)

# impervious surface
Map.addLayer(gisa, {}, "GISA 1972–2019")

# wetlands
wetlands_vis = {
    "min": 180,
    "max": 187,
    "palette": [
        "#CCCCCC",
        "#0000FF",
        "#006400",
        "#00FF00",
        "#00FFFF",
        "#CC99FF",
        "#556B2F",
        "#FFFF99",
        "#D2B48C",
    ],
}
Map.addLayer(wetlands.mask(wetlands.neq(0)), wetlands_vis, "GWL FCS30 Wetlands")

# ntl_vis = {
#     "opacity": 1,
#     "bands": ["avg_rad"],
#     "min": 1,
#     "max": 32,
#     "palette": ["584d9f", "9c79c1", "c98cbe", "f2d192", "e2ee82"],
# }

# Map.addLayer(ntl_composite.mask(ntl_composite.neq(0)), ntl_vis, "NTL VIIRS Latest")


# Create a monochromatic red color palette (light to dark)
# Using 7 shades from light pink to dark red
mono_red_palette = [
    "#FFEBEE",
    "#FFCDD2",
    "#EF9A9A",
    "#E57373",
    "#EF5350",
    "#F44336",
    "#B71C1C",
]

# Add to map with improved visualization parameters
vulnerability_vis = {
    "bands": ["vulnerability"],
    "min": 0,
    "max": 1,
    "palette": mono_red_palette,
}

Map.addLayer(
    vulnerability_smoothed,
    vulnerability_vis,
    "Socioeconomic Vulnerability (Log-Scaled)",
)

# rivers and subbasins
# basins
Map.addLayer(
    intersecting_basins.style(
        **{"color": "black", "fillColor": "00000000", "width": 1}
    ),
    {},
    "Intersecting Basins",
)
Map.addLayer(river_atlas, {"color": "#809fff"}, "HydroATLAS RiverATLAS v1.0")

Map

Map(center=[-34.963839822523525, -58.01607980918299], controls=(WidgetControl(options=['position', 'transparen…

In [ ]:
# necessary layers:
# - cuencas
# - rios
# admin units
# basemaps (satellite, positron)

### MAPS
# - latest pop
# - latest vulnerability (NLDI)
# - current wetlands
# current impervious

# do I need land cover? I might ignore this for now--maybe a bonus, but I'm not so sure

### Graphs
# population per cuenca per year, 2000 to present (more or less) -- world pop
# vulnerability per cuenca per year, 2000 to present (more or less) -- from NLDI, at smallest possible admin level